### **APP DE CONEXIÓN BASE DE DATOS**

---

In [21]:
# Importación de librerias utilizadas para la conección y tratamiento de la información
# Conección
from sqlalchemy import create_engine
from sqlalchemy.engine import URL 
import pypyodbc as odbc
# Tratamiento de información
import pandas as pd
import pandasql as ps
import numpy as np
# Tratamiento de fechas
from datetime import date
# Usuarios y contraseñas protegidas
import getpass

In [22]:
# Solicitar usuario y contraseña de acceso
SERVER_NAME = input('NOMBRE DEL SERVIDOR: ')
DATABASE_NAME = input('NOMBRE DE DATABASE: ')
db = input('NOMBRE TABLA/VISTA: ')
UID = input('Ingresa tu usuario: ')
PWD = getpass.getpass('Ingresa tu contraseña: ')

In [23]:
# Variable de login
connection_string = f'''
Driver={{SQL Server}};
Server={SERVER_NAME};
Database={DATABASE_NAME};
Uid={UID};
Pwd={PWD};
'''

In [24]:
# Variable SQL: Consulta SQL para extracción de data
# Filtro de fecha: WHERE CONVERT(DATE, fecha, 103) >= CONVERT(DATE, GETDATE()-1, 103)
# Código 103 corresponde al formateo de fecha DD/MM/YYYY
sql_query = f'''
SELECT *
FROM {db} 
'''

In [25]:
# Script de conexión con base de datos y servidor
connection_url = URL.create('mssql+pyodbc', query={'odbc_connect': connection_string})
engine = create_engine(connection_url, module=odbc)

In [26]:
# Script de descarga de data de la base de datos de acuerdo a la consulta SQL
df = pd.read_sql_query(sql_query, engine)

---

#### **ANÁLISIS DE DATA**

Funciones personalizadas

In [27]:
# Funciones de formateo de registros
# Formeteo de fecha
def date_formatting(fecha):
    date = fecha
    if type(date) == str:
        date = pd.to_datetime(date)
    result = date.strftime('%d-%m-%Y')
    return result

# Formateo de numeros
def num_formatting(numero):
    return '{:,.2f}'.format(numero)

# Proceso de DataFrame a travez de pandasql as ps para realizar consultas sql
def show_query(query, df):
    result = ps.sqldf(query, locals())
    return result

Datos generales del DataFrame "df"

In [28]:
# Variables de la base de datos
fecha_min = date_formatting(df['fecha'].min())
fecha_max = date_formatting(df['fecha'].max())
cuenta_df = num_formatting(df['fecha'].nunique())
cuenta_total_registros = num_formatting(df['fecha'].count())
cuenta_registros_df_max = num_formatting((df['fecha'] == df['fecha'].max()).sum())

In [29]:
# Visualización de información "df"
print("Datos de Generales")
print(f"Tabla: {db}")
print('------------------------------------------------------------')
print(f"Fecha mínima:       {fecha_min}")
print(f"Fecha máxima:       {fecha_max}")
print(f"Bases de datos:     {cuenta_df} Bases de datos")
print(f"Registos totales:   {cuenta_total_registros} en {cuenta_df} DB")
print(f"Registos ultima DB: {cuenta_registros_df_max} regristros de fecha {fecha_max}")

Datos de Generales
Tabla: Finanza_Score_Cartera_tbl
------------------------------------------------------------
Fecha mínima:       03-07-2023
Fecha máxima:       17-01-2024
Bases de datos:     185.00 Bases de datos
Registos totales:   2,892,837.00 en 185.00 DB
Registos ultima DB: 15,934.00 regristros de fecha 17-01-2024


* Análisis exploratorio

In [30]:
query = '''
SELECT * 
FROM df
'''
df_query = show_query(query, df)

In [31]:
# Variables de la base de datos
fecha_min_df = date_formatting(df_query['fecha'].min())
fecha_max_df = date_formatting(df_query['fecha'].max())
cuenta_df_df = num_formatting(df_query['fecha'].nunique())
cuenta_total_registros_df = num_formatting(df_query['fecha'].count())
cuenta_registros_df_max_df = num_formatting((df_query['fecha'] == df_query['fecha'].max()).sum())

In [32]:
# Visualización de información "df_query"
print("Datos de control de la tabla")
print(f"Tabla: DataFrame con Sub-consulta SQL a {db}")
print('------------------------------------------------------------')
print(f"Fecha mínima:       {fecha_min_df}")
print(f"Fecha máxima:       {fecha_max_df}")
print(f"Bases de datos:     {cuenta_df_df} Bases de datos")
print(f"Registos totales:   {cuenta_total_registros_df} en {cuenta_df_df} DB")
print(f"Registos ultima DB: {cuenta_registros_df_max_df} regristros de fecha {fecha_max_df}")

Datos de control de la tabla
Tabla: DataFrame con Sub-consulta SQL a Finanza_Score_Cartera_tbl
------------------------------------------------------------
Fecha mínima:       03-07-2023
Fecha máxima:       17-01-2024
Bases de datos:     185.00 Bases de datos
Registos totales:   2,892,837.00 en 185.00 DB
Registos ultima DB: 15,934.00 regristros de fecha 17-01-2024


In [33]:
# Datos de la tabla
print(df_query.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2892837 entries, 0 to 2892836
Data columns (total 18 columns):
 #   Column           Dtype  
---  ------           -----  
 0   agencia          int64  
 1   prestamo         int64  
 2   fecha            object 
 3   codigooficial    int64  
 4   saldodolares     float64
 5   vigente          float64
 6   atrazado         float64
 7   vencido          float64
 8   ejecutado        float64
 9   prevision        float64
 10  estado           int64  
 11  moneda           int64  
 12  fechaincumplido  object 
 13  mora             float64
 14  oficial          object 
 15  nrosocio         int64  
 16  nombrecliente    object 
 17  saldodiferido    float64
dtypes: float64(8), int64(6), object(4)
memory usage: 397.3+ MB
None


In [34]:
# Descrición por columan de la tabla
print(df_query.describe())

            agencia      prestamo  codigooficial  saldodolares       vigente  \
count  2.892837e+06  2.892837e+06   2.892837e+06  2.892837e+06  2.207873e+06   
mean   2.193009e+01  9.981603e+06   4.919735e+02  6.617113e+03  7.451637e+03   
std    9.814789e+00  4.393651e+06   1.434137e+02  1.613412e+04  1.729192e+04   
min    1.000000e+01  1.181000e+03   4.100000e+01  0.000000e+00  0.000000e+00   
25%    1.400000e+01  1.009460e+07   4.280000e+02  1.147838e+03  1.438112e+03   
50%    2.200000e+01  1.159030e+07   5.130000e+02  2.613692e+03  2.953810e+03   
75%    2.900000e+01  1.249230e+07   6.050000e+02  6.017633e+03  6.491901e+03   
max    4.300000e+01  1.429103e+07   7.290000e+02  4.591837e+05  4.591837e+05   

            atrazado        vencido      ejecutado     prevision  \
count  296990.000000  243020.000000   22342.000000  2.892640e+06   
mean     6122.372150    3308.258835   16346.061203  3.364543e+02   
std     15428.275165    8657.358800   26250.771208  1.645746e+03   
min    